# Congruencia longitudinal en participación de empresas 4_3_01

| **Escenario**                                                         | **Condición evaluada**                                                         | **Resultado**    |
| --------------------------------------------------------------------- | ------------------------------------------------------------------------------ | ---------------- |
| **El declarante no tiene más de una declaración en el periodo**       | Sólo existe M₀ o Mₖ sin Mᵢ intermedias                                         | ⚪ **NO_APLICA**  |
| **Faltan datos esenciales**                                           | No existe fecha de posesión del cargo o no existen fechas de las declaraciones | ⚪ **SIN_DATO**   |
| **Empresa presente en Mᵢ pero ausente en Mᵢ₊₁ sin baja/modificación** | La empresa desaparece sin registro de “Baja”, “Modificar” o “Sin cambio”       | 🔴 **NO_CUMPLE** |
| **Empresa cambia nombre sin estar marcada como modificación**         | Nombre diferente pero sin estatus “Modificar”                                  | 🔴 **NO_CUMPLE** |
| **Todas las empresas evolucionan correctamente**                      |                                                                                |                  |


In [12]:
import traceback
from pymongo import MongoClient, UpdateOne
from datetime import datetime
import unicodedata
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME


METRIC_ID = "4_3_01_CONGRUENCIA_PARTICIPACION_EMPRESAS"


# --------------------------------------
# Normalización
# --------------------------------------
def normalizar(v):
    if not v:
        return None
    v = str(v).strip().upper()
    v = unicodedata.normalize("NFD", v)
    v = "".join(c for c in v if unicodedata.category(c) != "Mn")
    v = v.replace('"', "").replace("'", "")
    while "  " in v:
        v = v.replace("  ", " ")
    return v


# extraer con ruta
def extraer(doc, path):
    try:
        parts = path.split(".")
        cur = doc
        for p in parts:
            if isinstance(cur, dict):
                cur = cur.get(p)
            elif isinstance(cur, list) and len(cur) > 0:
                cur = cur[0].get(p)
            else:
                return None
        return cur
    except:
        return None


# --------------------------------------
# EVALUACIÓN LONGITUDINAL
# --------------------------------------
def evaluar(declaraciones):

    # Orden por fecha
    declaraciones_ordenadas = sorted(
        declaraciones,
        key=lambda d: d.get("fecha") or datetime.max
    )

    if len(declaraciones_ordenadas) <= 1:
        return "NO_APLICA"

    # Extraer listas de empresas por declaración
    empresas_por_decl = []

    for item in declaraciones_ordenadas:
        doc = item["doc"]
        participaciones = extraer(doc, "declaracion.intereses.participacion.participacion")

        nombres = set()
        if participaciones:
            if not isinstance(participaciones, list):
                participaciones = [participaciones]

            for p in participaciones:
                nombre = normalizar(p.get("nombreEmpresa"))
                if nombre:
                    nombres.add(nombre)

        empresas_por_decl.append(nombres)

    # Comparar E(M_i) → E(M_i+1)
    for i in range(len(empresas_por_decl) - 1):
        actual = empresas_por_decl[i]
        siguiente = empresas_por_decl[i + 1]

        for empresa in actual:
            if empresa not in siguiente:
                return "NO_CUMPLE"

    return "CUMPLE"


# --------------------------------------
# PROCESADOR PRINCIPAL
# --------------------------------------
def procesar_metrica():
    print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    src = db[SOURCE_COLLECTION_NAME]
    tgt = db[METRICS_COLLECTION_NAME]

    # 1️⃣ obtener lista de todos los CURPs distintos (muy liviano)
    print("📌 Obteniendo lista de CURPs distintos...")
    curps = src.distinct("declaracion.datosGenerales.curp")

    total = 0
    ops = []

    for curp in curps:
        if not curp:
            continue

        # 2️⃣ obtener TODAS las declaraciones del declarante
        docs = list(src.find(
            {"declaracion.datosGenerales.curp": curp},
            {"_id": 1, "declaracion": 1}     # sólo lo necesario
        ))

        preparadas = []
        for d in docs:
            fecha = extraer(d["declaracion"], "fechaFirma")

            if fecha:
                try:
                    fecha = datetime.fromisoformat(fecha.replace("Z", ""))
                except:
                    fecha = None

            preparadas.append({
                "fecha": fecha,
                "doc": d["declaracion"],
                "_id": d["_id"]
            })

        # 3️⃣ evaluar la métrica longitudinal
        resultado = evaluar(preparadas)

        # 4️⃣ guardar resultado en TODAS sus declaraciones
        for d in preparadas:
            ops.append(
                UpdateOne(
                    {"_id": d["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )
            total += 1

            if len(ops) == 2000:
                tgt.bulk_write(ops)
                ops.clear()
                print(f" > {total} documentos actualizados...")

    if ops:
        tgt.bulk_write(ops)

    print("\n✔ PROCESO COMPLETADO")
    print("Total de documentos actualizados:", total)
    client.close()
    print("Conexión cerrada.")

if __name__ == "__main__":
    procesar_metrica()




🚀 Procesando métrica 4_3_01_CONGRUENCIA_PARTICIPACION_EMPRESAS...

📌 Obteniendo lista de CURPs distintos...

✔ PROCESO COMPLETADO
Total de documentos actualizados: 0
Conexión cerrada.
